In [2]:
import pandas as pd


In [ ]:
from google.cloud import storage
   ...: import os
   ...:
   ...: client = storage.Client.from_service_account_json(
   ...:     os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
   ...: )
   ...:
   ...: bucket = client.bucket(os.getenv("GCP_BUCKET_NAME"))
   ...:
   ...: print("Listing files:")
   ...: for blob in bucket.list_blobs():
   ...:     print("-", blob.name)

/Users/loso/.pyenv/versions/3.10.6/envs/inkling/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Listing files:
- Wearable SSVEP Dataset/Impedance.mat
- Wearable SSVEP Dataset/Readme.pdf
- Wearable SSVEP Dataset/S001.mat
- Wearable SSVEP Dataset/S002.mat
- Wearable SSVEP Dataset/S003.mat
- Wearable SSVEP Dataset/S004.mat
- Wearable SSVEP Dataset/S005.mat
- Wearable SSVEP Dataset/S006.mat
- Wearable SSVEP Dataset/S007.mat
- Wearable SSVEP Dataset/S008.mat
- Wearable SSVEP Dataset/S009.mat
- Wearable SSVEP Dataset/S010.mat
- Wearable SSVEP Dataset/S011.mat
- Wearable SSVEP Dataset/S012.mat
- Wearable SSVEP Dataset/S013.mat
- Wearable SSVEP Dataset/S014.mat
- Wearable SSVEP Dataset/S015.mat
- Wearable SSVEP Dataset/S016.mat
- Wearable SSVEP Dataset/S017.mat
- Wearable SSVEP Dataset/S018.mat
- Wearable SSVEP Dataset/S019.mat
- Wearable SSVEP Dataset/S020.mat
- Wearable SSVEP Dataset/S021.mat
- Wearable SSVEP Dataset/S022.mat
- Wearable SSVEP Dataset/S023.mat
- Wearable SSVEP Dataset/S024.mat
- Wearable SSVEP Dataset/S025.mat
- Wearable SSVEP Dataset/S026.mat
- Wearable SSVEP Dataset/S

In [7]:
from google.cloud import storage
import os

client = storage.Client.from_service_account_json(
    os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
)

bucket = client.bucket(os.getenv("GCP_BUCKET_NAME"))

blob = bucket.blob("Wearable SSVEP Dataset/S001.mat")
blob.download_to_filename("S001.mat")

print("Downloaded S001.mat")


Downloaded S001.mat


In [13]:
import numpy as np

In [8]:
import scipy.io as sio

mat = sio.loadmat("S001.mat")

In [10]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'data'])

In [18]:
import os
import re
import io

import numpy as np
import pandas as pd
import scipy.io as sio
from google.cloud import storage

client = storage.Client.from_service_account_json(
    os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
)
bucket = client.bucket(os.getenv("GCP_BUCKET_NAME"))

prefix = "Wearable SSVEP Dataset/"   # folder path inside the bucket

all_dfs = []

for blob in bucket.list_blobs(prefix=prefix):
    name = blob.name  # e.g. "Wearable SSVEP Dataset/S001.mat"

    # keep only S###.mat files
    m = re.search(r"(S\d+)\.mat$", name)
    if not m:
        continue

    subject_id = m.group(1)  # "S001", ...

    print("Loading", name)

    # download into memory
    data_bytes = blob.download_as_bytes()
    mat = sio.loadmat(io.BytesIO(data_bytes))

    data = mat["data"]   # shape: (12, 10, 2, 710, 8)

    # ---- reshape to 2D: (rows, channels) ----
    n1, n2, n3, n_samples, n_channels = data.shape

    # combine first 4 dims (12*10*2*710) into one long axis
    signals = data.reshape(-1, n_channels)   # shape: (n1*n2*n3*n_samples, n_channels)

    # make dataframe: one col per channel
    ch_cols = [f"ch{i+1}" for i in range(n_channels)]
    df = pd.DataFrame(signals, columns=ch_cols)

    # add subject + a generic index within subject
    df["subject"] = subject_id
    df["sample_idx"] = np.arange(len(df))

    all_dfs.append(df)

# ---- combine all subjects ----
all_data = pd.concat(all_dfs, ignore_index=True)

print(all_data.head())
print(all_data.shape)


Loading Wearable SSVEP Dataset/S001.mat
Loading Wearable SSVEP Dataset/S002.mat
Loading Wearable SSVEP Dataset/S003.mat
Loading Wearable SSVEP Dataset/S004.mat
Loading Wearable SSVEP Dataset/S005.mat
Loading Wearable SSVEP Dataset/S006.mat
Loading Wearable SSVEP Dataset/S007.mat
Loading Wearable SSVEP Dataset/S008.mat
Loading Wearable SSVEP Dataset/S009.mat
Loading Wearable SSVEP Dataset/S010.mat
Loading Wearable SSVEP Dataset/S011.mat
Loading Wearable SSVEP Dataset/S012.mat
Loading Wearable SSVEP Dataset/S013.mat
Loading Wearable SSVEP Dataset/S014.mat
Loading Wearable SSVEP Dataset/S015.mat
Loading Wearable SSVEP Dataset/S016.mat
Loading Wearable SSVEP Dataset/S017.mat
Loading Wearable SSVEP Dataset/S018.mat
Loading Wearable SSVEP Dataset/S019.mat
Loading Wearable SSVEP Dataset/S020.mat
Loading Wearable SSVEP Dataset/S021.mat
Loading Wearable SSVEP Dataset/S022.mat
Loading Wearable SSVEP Dataset/S023.mat
Loading Wearable SSVEP Dataset/S024.mat
Loading Wearable SSVEP Dataset/S025.mat


In [22]:
len(all_data.columns)

14

In [23]:
all_data.head()

,ch1,ch2,ch3,ch4,ch5,ch6,ch7,ch8,ch9,ch10,ch11,ch12,subject,sample_idx
0,-52325.520058,-56806.910444,-58424.819018,-59719.476683,-57667.363147,-54922.122057,-56358.355663,-54407.137895,-56100.774175,-62036.994817,-56265.908853,-56615.266598,S001,0
1,-44545.817050,-43069.395010,-42243.811026,-44446.754125,-41837.098708,-43248.924211,-40779.101299,-45746.686801,-45701.133949,-44418.859149,-43711.024148,-41958.871005,S001,1
2,-31622.530915,-34708.949012,-29636.265614,-33338.921269,-30578.972732,-32454.373387,-28531.731876,-29395.090306,-33787.520754,-29646.502712,-33290.105062,-32036.932232,S001,2
3,-18627.450980,-20911.396796,-18978.060423,-18118.054755,-18088.550454,-18863.574795,-18829.644849,-21339.298567,-19362.644515,-19725.815637,-16480.923681,-20240.710392,S001,3
4,-8561.522279,-10392.487669,-9935.707446,-12526.408584,-12226.582302,-12468.964605,-8784.592675,-11196.256351,-11033.356847,-8246.496811,-10760.218546,-11024.952592,S001,4
